In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from itertools import combinations
from sklearn.decomposition import PCA

In [2]:
ph=1.5574E-4
times=[1,2,3,6,7,9,13,16,21,24,31] 
protien="FAS_MOUSE" 

pxt=[(x) for x in list(range(3,50,1))]
neh=range(1,120)

In [72]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/"
protien="FAS_MOUSE" 
time=31
peptides=[x.split('_1_')[0] for x in os.listdir(path) if ((".csv" in x) and ("_1_" in x)) ]
peptides=[x  for x in peptides if 'A' in x]
all_AAs=np.unique([c for c in ("".join(sorted([x.replace('_','').replace('2','').replace('3','') for x in peptides])).lower())])
peptidesinfo=dict()

In [73]:
def charcount(pepSeq):
    res={key: 0 for key in ['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p',
       'q', 'r', 's', 't', 'v', 'w', 'y']}     
    for c in pepSeq.lower():
        if c.isalpha():
            res[c]+=1
    return res

In [74]:
for k in range(len(peptides)):
    data=pd.read_csv(f"C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/{peptides[k]}_{time}_{protien}.csv")
    data["pxt_neh"]=data.NEH*data.pxt
    data=data[["NEH","pxt","RMSE","pxt_neh","T_NEH","NH"]] 
    minval=min(data.RMSE)
    data=data[data.RMSE==minval]
    
    peptidesinfo[peptides[k]]={"pxt_neh":float(data.pxt_neh),"T_NEH":float(data.T_NEH),"Seq":peptides[k],
                           "charcount":charcount(peptides[k])}



In [98]:
coefficients=[]
for k in range(1,len(peptides)):
    pep1=peptidesinfo[peptides[0]]
    pep2=peptidesinfo[peptides[k]]
    r=pep1["pxt_neh"]/pep2["pxt_neh"]
    coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
    if(len(coefficients)==len(all_AAs)-1):
        break
print(len(coefficients))

12


In [99]:
for k in range(len(peptides)-2,1,-1):
    pep1=peptidesinfo[peptides[len(peptides)-1]]
    pep2=peptidesinfo[peptides[k]]
    r=pep1["pxt_neh"]/pep2["pxt_neh"]
    coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
    if(len(coefficients)==len(all_AAs)-1):
        break

In [100]:
coefficients=np.array(coefficients)
sol=list(np.linalg.solve(coefficients[:,1:],-4*coefficients[:,0])) 
sol.insert(0,4)
sol

[4,
 -2.875357810464157,
 0.17472699704094732,
 -5.809812993709621,
 -2.165444372241736,
 0.10356976676795178,
 -4.012226126582858,
 -3.8153357815702265,
 -4.166094893390312,
 0.9225220791243225,
 1.9768654719947327,
 1.2367352625828807,
 3.266565478899335,
 0.5964448939714654,
 -0.3111555318538495,
 -2.6555093816810773,
 -0.41865866896053816,
 4.165063920169344,
 3.3362505388637183,
 -2.8149858366114735]

In [110]:
# pep1=peptidesinfo[peptides[7]]
# p_neh=0
# for index,c in enumerate(all_AAs):
#     p_neh+=pep1["charcount"][c]*sol[index]
# p_neh

for i in range (len(peptides)):
    pep1=peptidesinfo[peptides[i]]
    p_neh=0
    for index,c in enumerate(all_AAs):
#         print(c,pep1["charcount"][c],pep1["charcount"][c]*sol[index])
        p_neh+=(pep1["charcount"][c]*sol[index])
    print(p_neh,"\t",pep1["T_NEH"])

-1.0480505352461478e-13 	 23.0
-1.6342482922482304e-13 	 37.0
-1.162403506782539e-13 	 26.0
-1.376676550535194e-13 	 36.0
-1.5987211554602254e-13 	 40.0
-1.0302869668521453e-13 	 24.0
-1.4210854715202004e-13 	 33.0
-1.4210854715202004e-13 	 33.0
-1.509903313490213e-13 	 39.0
-7.993605777301127e-14 	 19.0
-7.993605777301127e-14 	 19.0
-8.393286066166183e-14 	 22.0
-1.971756091734278e-13 	 38.0


In [86]:
[int(peptidesinfo[key]["T_NEH"]) for key in peptidesinfo.keys()]

[23, 37, 26, 36, 40, 24, 33, 33, 39, 19, 19, 22, 38]

In [87]:
pd.DataFrame(coefficients)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.352941,1.0,-0.294118,-0.294118,-0.647059,0.000000,0.000000,0.000000,-0.294118,1.705882,-0.647059,-0.294118,-1.294118,-1.294118,0.000000,1.000000,1.352941,0.352941,0.000000,0.000000
1,1.083334,1.0,-0.833332,1.000000,-1.833332,0.000000,0.000000,0.000000,1.000000,2.083334,0.000000,0.083334,-0.916666,-0.916666,-0.916666,-0.833332,1.083334,1.000000,0.000000,0.000000
2,0.503401,1.0,1.000000,0.251701,-0.748299,-2.244898,0.000000,0.000000,1.000000,3.000000,-0.748299,-1.244898,0.000000,-0.748299,-0.748299,0.251701,0.503401,1.000000,0.000000,-0.748299
3,1.333333,1.0,0.333333,-0.333333,-0.666667,0.000000,-0.666667,0.000000,1.000000,2.333333,0.000000,1.000000,-2.666667,0.000000,-0.666667,-1.000000,1.333333,-0.333333,0.000000,0.000000
4,1.023957,1.0,-0.952085,0.023957,0.000000,0.000000,0.000000,0.000000,0.023957,2.023957,0.000000,1.000000,0.000000,-0.976043,-0.976043,0.023957,-0.928128,-0.952085,0.000000,0.000000
5,1.290323,1.0,0.290323,0.290323,0.000000,-0.709677,0.000000,-0.709677,0.290323,-1.967742,0.000000,0.290323,-1.419355,-0.709677,0.000000,-0.419355,2.000000,1.000000,0.000000,0.000000
6,1.275839,1.0,0.275839,0.275839,0.000000,-0.724161,0.000000,-0.724161,0.275839,-2.069124,0.000000,0.275839,-1.448321,-0.724161,0.000000,-0.448321,2.000000,1.000000,0.000000,0.000000
7,1.333333,1.0,1.000000,0.333333,0.000000,-1.333333,0.000000,-0.666667,1.000000,1.666667,0.000000,0.333333,0.000000,-1.333333,-0.666667,-1.666667,1.333333,-1.000000,0.000000,0.000000
8,0.741419,1.0,-0.258581,-0.258581,0.000000,-1.258581,0.000000,0.000000,-1.517162,1.741419,0.000000,1.000000,-1.258581,0.000000,0.000000,-0.258581,2.000000,-1.517162,0.000000,0.000000
9,0.705882,1.0,-0.294118,-0.294118,0.000000,-1.294118,0.000000,0.000000,-1.588235,1.705882,0.000000,1.000000,-1.294118,0.000000,0.000000,-0.294118,2.000000,-1.588235,0.000000,0.000000


In [92]:
for c in range( len(coefficients)):
    print(np.dot(coefficients[c],sol))

1.0447901827485734e-15
2.6645352591003757e-15
-1.4314034795590501e-15
1.9366037603307308e-15
-3.0419474626915746e-15
-1.7763568394002505e-15
-1.7763568394002505e-15
-2.6645352591003757e-15
-6.6643384862337595e-15
-1.149909787206458e-15
-4.643910028884266e-15
-2.8849790995995347e-15
-8.551260221584568e-16
8.881784197001252e-16
-4.440892098500626e-15
-1.7763568394002505e-15
2.6645352591003757e-15
0.0
-1.7763568394002505e-15


In [93]:
pep1=peptidesinfo[peptides[0]]
pep2=peptidesinfo[peptides[1]]
r=pep1["pxt_neh"]/pep2["pxt_neh"]
r

0.6470588235294117

In [94]:
all_AAs

array(['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p',
       'q', 'r', 's', 't', 'v', 'w', 'y'], dtype='<U1')